In [10]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
from sklearn.preprocessing import MinMaxScaler

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [12]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):

    if num_layers == 2:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(144, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )

    else:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(120, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )


    return network

In [13]:
PATH = r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\good models\lemon-sweep-1_990.pt"
checkpoint = torch.load(PATH)
# 
# epoch,model_sd,optim_sd,loss_sd = 


In [14]:
model = build_network(256 ,64,0.3,2)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

Sequential(
  (0): Linear(in_features=144, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=64, out_features=8, bias=True)
  (7): Softmax(dim=1)
)

In [15]:
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
pred_inputs_file = open( 'prediction_inputs.npy', 'rb')
data= pickle.load(pred_inputs_file)
race_ids,inputs,names = zip(*data)
for (i,j) in zip(race_ids,names):
    print(i,j)

832364915 ['BARRY ALLAN', 'WALDEN BALE', 'BROOKLYN STAR', 'ISHMY BELLE', 'SCOTT ME', 'DALLAS JANE', 'DELINQUENT', 'ESPECIAL HARVEY']
832364916 ['FLYING JARRAD', "OSWALD'S DELIGHT", 'DOWN UNDER MISS', 'SCOTT MONSTER', 'MISS TINGLES', 'FORWARD STEP', 'RUSHWORTH', 'CONFOUND']
832364914 ['PATCHWORK PANDA', 'NO JEWELS', 'TWENTY TWO BLACK', 'CHAMP DURAN', "IT'S LUIGI", 'AEROPLANE LEXI', 'MISS CHARLO', 'COSMETIC ROSE']
832364913 ['GROOMSMAN', 'AUSSIE SHOWGIRL', 'CLARETOWN MAC', 'LIKE A LORRY', 'HOT CROSS', 'ROJO SILVER FOX', 'ENSEMBLE', 'WHO TOLD JOHN']
832364907 ['DISORDER', 'WINDY BISCUIT', 'COOL WHIP', 'SMOOTH CRUISER', 'READY TO LAUNCH', 'BRAVE LAD', 'MCIVOR BONNIE', 'BENNY JET']
832364918 ['WOW STAR', 'FOXTROT TANGO', 'PRINCESS WARRIOR', 'DAINTREE LAD', 'JOEY MAC', "I'M NOT BUDGING", 'ITCHY KAALA', "REZZIE'S SHOUT"]
832364908 ['GIPPSLAND BOY', 'SPEARHEAD JOE', 'AMERICAN STAR', 'ESPECIAL SPEED', 'COLLEGE BOY', 'MISS BELLE', 'REMNANTS', 'MAXIMUM ZEAL']
832364917 ['JOHNNY ONE THONG', "IRENE

In [16]:
scaler_file = open(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\normalizer.npy", "rb")
scaler = pickle.load(scaler_file)
scaler
Xn = scaler.transform(X)
Xn = torch.tensor(Xn).float()

In [17]:
inputs_df = pd.DataFrame(inputs)
inputs_df.fillna(value=-1, inplace=True)
inputs = inputs_df.values.tolist()
inputs = [[float(i) for i in j] for j in inputs]

X = torch.Tensor(np.array([i for i in inputs]))
X.shape
Xn = scaler.transform(X)
Xn = torch.tensor(Xn).float()
Xn

tensor([[0.0000, 1.0000, 0.0000,  ..., 0.1206, 0.9938, 0.1186],
        [0.0000, 1.0000, 0.0000,  ..., 0.1265, 0.9947, 0.1018],
        [0.0000, 1.0000, 0.0000,  ..., 0.1202, 0.9952, 0.1998],
        ...,
        [0.0000, 1.0000, 0.0000,  ..., 0.1272, 0.9946, 0.1322],
        [0.0000, 1.0000, 0.0000,  ..., 0.1156, 0.9941, 0.0227],
        [0.0000, 1.0000, 0.0000,  ..., 0.1105, 0.9948, 0.2537]])

In [18]:
REBUILD_DATA = True


class GRV_pred:
    # class to store prediction data

    # reading data from pickle
    file = open(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATAprediction_inputs.npy", "rb")
    data = pickle.load(file)
    # seperate out classes from inputs
    raceIDs, inputs = zip(*data)
    # removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1, inplace=True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]

    # data
    pred_data = []

    def make_pred_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.inputs[i]) == 120:
                self.pred_data.append(
                    [
                        np.array(self.inputs[i]),
                    ]
                )
            else:
                adjustedList = self.classes[i] + ([8] * (8 - len(self.classes[i])))
                adjustedListP = self.prices[i] + ([0] * (8 - len(self.prices[i])))
                adjustedListM = self.margins[i] + ([100] * (8 - len(self.margins[i])))
                adjustedListSP = self.margins[i] + ([0] * (8 - len(self.betfairSP[i])))
                self.pred_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(adjustedList),
                        np.array(adjustedListP),
                        np.array(adjustedListM),
                        np.array(adjustedListSP)
                    ]
                )
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save("pred_data.npy", self.pred_data)
        print("excluded = ", excluded)


if REBUILD_DATA:
    grv = GRV_pred()
    grv.make_pred_data()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\DATAprediction_inputs.npy'

In [19]:


with torch.no_grad():
    model.eval()
    outputs = model(Xn)
    print()
    print(torch.argmax(outputs,1).tolist())
    outs = torch.argmax(outputs,1).tolist()
    for i,j in enumerate(names):
        print(j[outs[i]])



[6, 6, 7, 5, 7, 5, 7, 2, 1, 5, 5, 4]
DELINQUENT
RUSHWORTH
COSMETIC ROSE
ROJO SILVER FOX
BENNY JET
I'M NOT BUDGING
MAXIMUM ZEAL
JAYLA STRIKE
HEARTS DENIED
JOYOUS LEGACY
DEVIL'S WATER
VORTEX
